In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [3]:
sentence = "This is Natural Language Processing"
sentence_vector = sentence.split()
sentence_vector

['This', 'is', 'Natural', 'Language', 'Processing']

In [ ]:
# One Hot Encoding Function

def encoding(sentence_vector):
    encoding_list = []
    for i in range(n):
        l = [0]*n
        l[i] = 1
        encoding_list.append(l)
    return encoding_list

In [5]:
encoding_list = encoding(sentence_vector)
print(encoding_list)

NameError: name 'n' is not defined

In [ ]:
n = len(encoding_list)
print(n)

5


In [ ]:
# HardCoded CBOW with wondow size = 2

context_window = 2
context_dict = {}
avg_context = {}
for i in range(n):
    p = np.array([0]*n)
    count = 0
    context_dict[sentence_vector[i]] = []
    if i-2 >=0:
        context_dict[sentence_vector[i]].append(sentence_vector[i-2])
        p = p + np.array(encoding_list[i-2])
        count+=1
    if i-1 >= 0:
        context_dict[sentence_vector[i]].append(sentence_vector[i-1])
        p = p + np.array(encoding_list[i-1])
        count+=1
    if i+1 < n:
        context_dict[sentence_vector[i]].append(sentence_vector[i+1])
        p = p + np.array(encoding_list[i+1])
        count+=1
    if i+2 < n:
        context_dict[sentence_vector[i]].append(sentence_vector[i+2])
        p = p + np.array(encoding_list[i+2])
        count+=1
    avg_context[sentence_vector[i]] = p / count                         # Calculate the average context vector

for k,v in context_dict.items():
    print(f"Target Word: {k}\t\tContext Words: {v}")
print()
for k,v in avg_context.items():
    print(f"Target Word: {k}\t\tAverage Context: {v}")

Target Word: This		Context Words: ['is', 'Natural']
Target Word: is		Context Words: ['This', 'Natural', 'Language']
Target Word: Natural		Context Words: ['This', 'is', 'Language', 'Processing']
Target Word: Language		Context Words: ['is', 'Natural', 'Processing']
Target Word: Processing		Context Words: ['Natural', 'Language']

Target Word: This		Average Context: [0.  0.5 0.5 0.  0. ]
Target Word: is		Average Context: [0.33333333 0.         0.33333333 0.33333333 0.        ]
Target Word: Natural		Average Context: [0.25 0.25 0.   0.25 0.25]
Target Word: Language		Average Context: [0.         0.33333333 0.33333333 0.         0.33333333]
Target Word: Processing		Average Context: [0.  0.  0.5 0.5 0. ]


In [ ]:
# CBOW Model Architecture and Definition

class Model:
    def __init__(self, N, dim = 2):
        self.w = torch.rand(size = (N,2), dtype = torch.float32, requires_grad = True)
        self.wt = torch.rand(size = (2,N), dtype = torch.float32, requires_grad = True)
    
    def feed_forward(self, avg_emb):
        avg_emb = avg_emb.view(1, -1)
        i1 = avg_emb @ self.w
        i2 = i1 @ self.wt
        y_pred = torch.softmax(i2, dim = 1)
        return y_pred

In [ ]:
N = 5
e = []
model = Model(N, dim = 2)
for k,v in avg_context.items():
    t = model.feed_forward(torch.tensor(v, dtype = torch.float32))   # Feed Foward the computed Tokens to generate Embeddings
    e.append(t)
    print(f"Final Encoding: {t}")

Final Encoding: tensor([[0.2503, 0.2217, 0.1695, 0.2076, 0.1509]], grad_fn=<SoftmaxBackward0>)
Final Encoding: tensor([[0.2527, 0.2177, 0.1610, 0.2147, 0.1539]], grad_fn=<SoftmaxBackward0>)
Final Encoding: tensor([[0.2495, 0.2105, 0.1537, 0.2232, 0.1630]], grad_fn=<SoftmaxBackward0>)
Final Encoding: tensor([[0.2460, 0.2150, 0.1642, 0.2146, 0.1602]], grad_fn=<SoftmaxBackward0>)
Final Encoding: tensor([[0.2439, 0.2146, 0.1661, 0.2137, 0.1617]], grad_fn=<SoftmaxBackward0>)


In [ ]:
print(sentence_vector)

['This', 'is', 'Natural', 'Language', 'Processing']


In [ ]:
embeddings = [
    [0.2414, 0.2223, 0.1329, 0.2346, 0.1688],
    [0.2187, 0.2059, 0.1642, 0.2328, 0.1785],
    [0.2302, 0.2154, 0.1485, 0.2305, 0.1754],
    [0.2447, 0.2303, 0.1322, 0.2173, 0.1754],
    [0.2207, 0.2126, 0.1651, 0.2155, 0.1862],
]
input_seq = torch.tensor(embeddings).unsqueeze(0)

In [ ]:
# Testing with LSTM

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        last_hidden = out[:, -1, :]
        return self.fc(last_hidden)

In [ ]:
input_dim = 5
hidden_dim = 16
output_dim = 3

model = LSTMModel(input_dim, hidden_dim, output_dim)
output = model(input_seq)
print(output)

tensor([[ 0.1594, -0.1142, -0.1802]], grad_fn=<AddmmBackward0>)


In [ ]:
probs = torch.softmax(output, dim=1)
print(probs)

tensor([[0.4044, 0.3076, 0.2880]], grad_fn=<SoftmaxBackward0>)


### Next Word Prediction using CBOW Embeddings

In [ ]:
data = [
    "I like deep learning",
    "I like natural language processing",
    "Natural language processing is fun",
    "Deep learning is powerful",
    "I enjoy learning new things",
    "Language models are very useful",
    "PyTorch makes building models easy",
    "Deep learning and NLP go hand in hand",
    "This is a simple example",
    "We are training a neural network",
    "Neural networks can learn patterns",
    "This example shows how CBOW works",
    "The model learns word embeddings",
    "Embeddings capture semantic meaning",
    "Context matters in language modeling",
    "This dataset is still small but better",
    "You can use larger datasets for better results",
    "Training with more data improves performance",
    "Next word prediction is a common NLP task",
    "Let's build something cool with NLP"
]


In [ ]:
tokenized_sentences = []
for x in data:
    tokenized_sentences.append(x.lower().split())
all_words = []
for x in tokenized_sentences:
    for y in x:
        all_words.append(y)
all_words = sorted(set(all_words))
all_words


['a',
 'and',
 'are',
 'better',
 'build',
 'building',
 'but',
 'can',
 'capture',
 'cbow',
 'common',
 'context',
 'cool',
 'data',
 'dataset',
 'datasets',
 'deep',
 'easy',
 'embeddings',
 'enjoy',
 'example',
 'for',
 'fun',
 'go',
 'hand',
 'how',
 'i',
 'improves',
 'in',
 'is',
 'language',
 'larger',
 'learn',
 'learning',
 'learns',
 "let's",
 'like',
 'makes',
 'matters',
 'meaning',
 'model',
 'modeling',
 'models',
 'more',
 'natural',
 'network',
 'networks',
 'neural',
 'new',
 'next',
 'nlp',
 'patterns',
 'performance',
 'powerful',
 'prediction',
 'processing',
 'pytorch',
 'results',
 'semantic',
 'shows',
 'simple',
 'small',
 'something',
 'still',
 'task',
 'the',
 'things',
 'this',
 'training',
 'use',
 'useful',
 'very',
 'we',
 'with',
 'word',
 'works',
 'you']

In [42]:
def encoding(vocab_size):
    encoding_list = []
    for i in range(vocab_size):
        l = [0]*vocab_size
        l[i] = 1
        encoding_list.append(l)
    return encoding_list

n = len(all_words)
encoding_list = encoding(n)
encoding_list

[[1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [43]:
word_to_index = {word: i for i, word in enumerate(all_words)}
index_to_word = {i: word for word, i in word_to_index.items()}

In [44]:
context_window = 2
context_dict = {}
avg_context = {}

sentence_vector = all_words
word_indices = [word_to_index[word] for word in sentence_vector]

for i in range(len(sentence_vector)):
    p = np.array([0]*n)
    count = 0
    context_dict[sentence_vector[i]] = []

    if i-2 >= 0:
        context_dict[sentence_vector[i]].append(sentence_vector[i-2])
        p = p + np.array(encoding_list[word_to_index[sentence_vector[i-2]]])
        count += 1
    if i-1 >= 0:
        context_dict[sentence_vector[i]].append(sentence_vector[i-1])
        p = p + np.array(encoding_list[word_to_index[sentence_vector[i-1]]])
        count += 1
    if i+1 < len(sentence_vector):
        context_dict[sentence_vector[i]].append(sentence_vector[i+1])
        p = p + np.array(encoding_list[word_to_index[sentence_vector[i+1]]])
        count += 1
    if i+2 < len(sentence_vector):
        context_dict[sentence_vector[i]].append(sentence_vector[i+2])
        p = p + np.array(encoding_list[word_to_index[sentence_vector[i+2]]])
        count += 1

    avg_context[sentence_vector[i]] = p / count

print("Context Dictionary:\n")
for k, v in context_dict.items():
    print(f"Target Word: {k}\tContext Words: {v}")

print("\nAverage Embeddings:\n")
for k, v in avg_context.items():
    print(f"Target Word: {k}\tAverage Context: {v}")

Context Dictionary:

Target Word: a	Context Words: ['and', 'are']
Target Word: and	Context Words: ['a', 'are', 'better']
Target Word: are	Context Words: ['a', 'and', 'better', 'build']
Target Word: better	Context Words: ['and', 'are', 'build', 'building']
Target Word: build	Context Words: ['are', 'better', 'building', 'but']
Target Word: building	Context Words: ['better', 'build', 'but', 'can']
Target Word: but	Context Words: ['build', 'building', 'can', 'capture']
Target Word: can	Context Words: ['building', 'but', 'capture', 'cbow']
Target Word: capture	Context Words: ['but', 'can', 'cbow', 'common']
Target Word: cbow	Context Words: ['can', 'capture', 'common', 'context']
Target Word: common	Context Words: ['capture', 'cbow', 'context', 'cool']
Target Word: context	Context Words: ['cbow', 'common', 'cool', 'data']
Target Word: cool	Context Words: ['common', 'context', 'data', 'dataset']
Target Word: data	Context Words: ['context', 'cool', 'dataset', 'datasets']
Target Word: dataset	C

In [ ]:
inputs = []
targets = []

for i in range(len(sentence_vector)):
    context_vec = avg_context[sentence_vector[i]]
    inputs.append(context_vec)
    targets.append(word_to_index[sentence_vector[i]])

inputs = torch.tensor(inputs, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.long)

In [ ]:
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, vocab_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

model = LSTMModel(input_size=n, hidden_size=32, vocab_size=n)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()
    output = model(inputs)
    loss = criterion(output, targets)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


def predict_next_word(word):
    with torch.no_grad():
        context_vec = avg_context[word]
        inp = torch.tensor(context_vec, dtype=torch.float32).unsqueeze(0)
        out = model(inp)
        pred_index = torch.argmax(out, dim=1).item()
        return index_to_word[pred_index]

predicted = predict_next_word("can")
predicted


Epoch 0, Loss: 2.7867104179790658e-08
Epoch 10, Loss: 2.941527554867207e-08
Epoch 20, Loss: 6.34750705330589e-08
Epoch 30, Loss: 4.334883030310266e-08
Epoch 40, Loss: 3.560796812962508e-08
Epoch 50, Loss: 2.7867104179790658e-08
Epoch 60, Loss: 2.941527554867207e-08
Epoch 70, Loss: 1.0217937784773312e-07
Epoch 80, Loss: 4.180065715786441e-08
Epoch 90, Loss: 5.108968892386656e-08
Epoch 100, Loss: 7.121593625925016e-08
Epoch 110, Loss: 7.74086217347758e-08
Epoch 120, Loss: 1.3933552089895329e-08
Epoch 130, Loss: 3.715613772214965e-08
Epoch 140, Loss: 3.096344869391032e-08
Epoch 150, Loss: 9.443851922696922e-08
Epoch 160, Loss: 8.979400689668182e-08
Epoch 170, Loss: 6.96677631140119e-08
Epoch 180, Loss: 5.5734208359581316e-08
Epoch 190, Loss: 6.96677631140119e-08
Epoch 200, Loss: 6.037872424258239e-08
Epoch 210, Loss: 2.7867104179790658e-08
Epoch 220, Loss: 3.560796812962508e-08
Epoch 230, Loss: 6.037872424258239e-08
Epoch 240, Loss: 5.728238150481957e-08
Epoch 250, Loss: 3.251162183914857

'can'